In [1]:
import pandas as pd
import torch
# /Users/arrao/Documents/Atchut/Book-PPML/Chapter6/simulated_transactions_csv3/transactions.csv 

url ="/Users/arrao/Documents/Atchut/Book-PPML/Chapter6/fraud_transactions.csv"
#url = "/Users/arrao/Documents/Atchut/Book-PPML/Chapter6/simulated_transactions_csv/2023-07-08.csv"
df_actual = pd.read_csv(url, sep=",")
df_actual.head()

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD
0,0,0,2023-02-01 00:43:37,901,8047,82,1
1,1,1,2023-02-01 01:20:13,2611,7777,15,0
2,2,2,2023-02-01 01:22:52,4212,3336,53,0
3,3,3,2023-02-01 01:26:40,1293,7432,59,0
4,4,4,2023-02-01 01:52:23,2499,1024,25,0


In [2]:
df_transactions = df_actual[['CUSTOMER_ID','TERMINAL_ID','TX_AMOUNT','TX_FRAUD']]
#df_transactions=df_transactions.head(10)
df_transactions=df_transactions.head(50000)
df_transactions

,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD
0,901,8047,82,1
1,2611,7777,15,0
2,4212,3336,53,0
3,1293,7432,59,0
4,2499,1024,25,0
...,...,...,...,...
49995,1541,3469,66,0
49996,489,3854,89,1
49997,4125,7519,12,0
49998,3360,5215,86,1


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

print("No of Fraud Transactions:", df_transactions['TX_FRAUD'].value_counts()[0])
print("No of Non Fraud Transactions:", df_transactions['TX_FRAUD'].value_counts()[1])

print('No Frauds', round(df_transactions['TX_FRAUD'].value_counts()[0]/len(df_transactions) * 100,2), '% of the dataset')
print('Frauds', round(df_transactions['TX_FRAUD'].value_counts()[1]/len(df_transactions) * 100,2), '% of the dataset')



X = df_transactions.drop('TX_FRAUD', axis=1)
y = df_transactions['TX_FRAUD']



No of Fraud Transactions: 37870
No of Non Fraud Transactions: 12130
No Frauds 75.74 % of the dataset
Frauds 24.26 % of the dataset


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



In [5]:
x_train = torch.FloatTensor(X_train.values)
x_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values)
y_test = torch.FloatTensor(y_test.values)

In [6]:
print("############# Data summary #############")
print(f"x_train has shape: {x_train.shape}")
print(f"y_train has shape: {y_train.shape}")
print(f"x_test has shape: {x_test.shape}")
print(f"y_test has shape: {y_test.shape}")
print("#######################################")

############# Data summary #############
x_train has shape: torch.Size([35000, 3])
y_train has shape: torch.Size([35000])
x_test has shape: torch.Size([15000, 3])
y_test has shape: torch.Size([15000])
#######################################


In [7]:
if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

Selected device is cpu


In [8]:
class FraudDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, y):
        'Initialization'
        self.x = x
        self.y = y

    def __len__(self):
        'Returns the total number of samples'
        return len(self.x)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample index
        if self.y is not None:
            return self.x[index].to(DEVICE), self.y[index].to(DEVICE)
        else:
            return self.x[index].to(DEVICE)

In [9]:
train_loader_params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 0}
test_loader_params = {'batch_size': 64,
          'num_workers': 0}

# Loaders

training_set = FraudDataset(x_train, y_train)

testing_set = FraudDataset(x_test, y_test)


train_loader = torch.utils.data.DataLoader(training_set, **train_loader_params)
test_loader = torch.utils.data.DataLoader(testing_set, **test_loader_params)

In [18]:
class SimpleFraudMLP(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.first_sec = torch.nn.Sequential(
                           torch.nn.Linear(3, 450),
                           torch.nn.ReLU(),
        )
        self.second_sec = torch.nn.Sequential(
                           torch.nn.Linear(450, 450),
                           torch.nn.ReLU(),
                           torch.nn.Linear(450, 1),
                           torch.nn.Sigmoid(),
        )
        
    def forward(self, x):
        return self.second_sec(self.first_sec(x))


In [11]:
fraud_nn_model = SimpleFraudMLP().to(DEVICE)

In [12]:
from torch import nn, optim
#loss_func = nn.CrossEntropyLoss()   
loss_func = torch.nn.BCELoss().to(DEVICE)
loss_func


BCELoss()

In [14]:
#optimizer = optim.Adam(fraud_nn_model.parameters(), lr = 0.01)  
optimizer = torch.optim.SGD(fraud_nn_model.parameters(), lr = 0.07)
optimizer


SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.07
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [15]:
fraud_nn_model.eval()

SimpleFraudMLP(
  (first_sec): Sequential(
    (0): Linear(in_features=3, out_features=450, bias=True)
    (1): ReLU()
  )
  (second_sec): Sequential(
    (0): Linear(in_features=450, out_features=450, bias=True)
    (1): ReLU()
    (2): Linear(in_features=450, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

In [16]:
def evaluate_model(model,generator,loss_func):
    model.eval()
    batch_losses = []
    for x_batch, y_batch in generator:
        # Forward pass
        y_pred = model(x_batch)
        # Compute Loss
        loss = loss_func(y_pred.squeeze(), y_batch)
        batch_losses.append(loss.item())
    mean_loss = np.mean(batch_losses)    
    return mean_loss
    
#evaluate_model(fraud_nn_model,testing_loader,loss_func)

In [53]:
def train(fraud_nn_mode,num_epochs):
    
    fraud_nn_model.train()
               
    for epoch in range(num_epochs):

         #for x_batch, y_batch in train_loader:
                
            output = fraud_nn_model(x_train)  
            
            print(output.squeeze())
            
            print(y_train)
            
            loss = loss_func(output.squeeze(), y_train)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            
            print(epoch, loss.item())

        
    pass
            
train (fraud_nn_model, 10)

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1.,  ..., 0., 0., 0.])
0 24.191429138183594
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1.,  ..., 0., 0., 0.])
1 24.191429138183594
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1.,  ..., 0., 0., 0.])
2 24.191429138183594
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1.,  ..., 0., 0., 0.])
3 24.191429138183594
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1.,  ..., 0., 0., 0.])
4 24.191429138183594
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1.,  ..., 0., 0., 0.])
5 24.191429138183594
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1.,  ..., 0., 0., 0.])
6 24.191429138183594
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SqueezeBackward0>)
tensor([1., 1., 1.,  ..., 0., 0., 0.])
7 24.

In [17]:
def accuracy(model, x, y):
    out = model(x)
    correct = torch.abs(y - out) < 0.5
    return correct.float().mean()

plain_accuracy = accuracy(fraud_nn_model, x_test, y_test)
print(f"Accuracy on plain test_set: {plain_accuracy}")

Accuracy on plain test_set: 0.244234099984169


In [30]:
class LR(torch.nn.Module):

    def __init__(self):
        super().__init__()
        
        self.lr = torch.nn.Sequential(
                           torch.nn.Linear(3,1),
        )
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out

n_features = x_train.shape[1]
model = LR()
# use gradient descent with a learning_rate=1
optim = torch.optim.SGD(model.parameters(), lr=1)
# use Binary Cross Entropy Loss
criterion = torch.nn.BCELoss()
# define the number of epochs for both plain and encrypted training
EPOCHS = 5

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        print(f"Loss at epoch {e}: {loss.data}")
    return model

model = train(model, optim, criterion, x_train, y_train)

ValueError: Using a target size (torch.Size([35000])) that is different to the input size (torch.Size([35000, 1])) is deprecated. Please ensure they have the same size.

In [16]:
import numpy as np
import time

n_epochs = 50
#Setting the model in training mode
fraud_nn_model.train()

#Training loop
start_time=time.time()
epochs_train_losses = []
epochs_test_losses = []

for epoch in range(n_epochs):
    train_loss=[]
    train_loss1=0
    for x_batch, y_batch in train_loader:
        
        fraud_nn_model.train()
        
        # Performing the forward pass on the current batch
        y_pred = fraud_nn_model(x_batch)
        # Computing the loss given the current predictions
        loss = loss_func(y_pred.squeeze(), y_batch)
        # Computing the gradients over the backward pass
        
        # Removing previously computed gradients
        optimizer.zero_grad()
        
        loss.backward()
        # Performing an optimization step from the current gradients
        optimizer.step()
        # Storing the current step's loss for display purposes
        train_loss.append(loss.item())
        
        train_loss1 += loss.item()*x_batch.size(0)
    avg_loss1 = train_loss1/len(train_loader.dataset)
    print("Batch Loss{} :", avg_loss1)
    
    #showing last training loss after each epoch
    epochs_train_losses.append(np.mean(train_loss))
    print('Epoch {}: train loss: {}'.format(epoch, np.mean(train_loss)))
    
    
    #evaluating the model on the test set after each epoch    
    val_loss = evaluate_model(fraud_nn_model,test_loader,loss_func)    
    epochs_test_losses.append(val_loss)
    print('test loss: {}'.format(val_loss))   
    print("")
  
    
training_execution_time=time.time()-start_time
training_execution_time

Batch Loss{} : 75.28534032679967
Epoch 0: train loss: 75.28690740177356
test loss: 75.60062057819772

Batch Loss{} : 75.8085714250837
Epoch 1: train loss: 75.80920279047825
test loss: 75.60062057819772

Batch Loss{} : 75.80857143205915
Epoch 2: train loss: 75.80757051847971
test loss: 75.60062057819772

Batch Loss{} : 75.80857142333984
Epoch 3: train loss: 75.80961085847788
test loss: 75.60062057819772

Batch Loss{} : 75.80857143380301
Epoch 4: train loss: 75.81001894042522
test loss: 75.60062057819772

Batch Loss{} : 75.80857143380301
Epoch 5: train loss: 75.80716245048006
test loss: 75.60062057819772

Batch Loss{} : 75.80857142857143
Epoch 6: train loss: 75.80838665447898
test loss: 75.60062057819772

Batch Loss{} : 75.80857143380301
Epoch 7: train loss: 75.81001894042522
test loss: 75.60062057819772

Batch Loss{} : 75.80857142857143
Epoch 8: train loss: 75.80838665447898
test loss: 75.60062057819772

Batch Loss{} : 75.80857142857143
Epoch 9: train loss: 75.80838665447898
test loss: 

111.07959914207458